In [ ]:
# Cell 1: Install Necessary Packages (Latest Stable Versions)
# Upgrading pip first and installing latest crewai/langchain versions.
# This aims for compatibility with the Colab environment.

!pip install --upgrade pip -q
!pip install -q crewai crewai[tools] langchain-google-genai python-dotenv google-api-python-client langchain-community

print("--- Packages Installed (Latest Stable Versions) ---")
# It's ESSENTIAL to restart the Colab Runtime after installing/changing package versions.
# This ensures the newly installed libraries are loaded correctly.
print("--- !!! IMPORTANT: Please RESTART the Colab Runtime now (Runtime -> Restart Session) !!! ---")
print("--- After restarting, re-run all cells starting from Cell 2. ---")

In [ ]:
# Cell 2: API Key Configuration (Using Direct Input - Option 2)
# Securely store and load your API keys for Gemini and Serper.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME

import os

print("--- Configuring API Keys using Direct Input (Option 2) ---")

# --- OPTION 1: Using Colab Secrets (Recommended for better security) ---
# To use this: comment out Option 2, uncomment these lines, and add
# 'GEMINI_API_KEY', 'SERPER_API_KEY' to Colab Secrets (key icon).
# from google.colab import userdata
# os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
# os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

# --- OPTION 2: Paste keys directly (Less Secure - Use with caution) ---
print("Setting keys directly using Option 2...")
# >>> ACTION REQUIRED: Uncomment the two lines below and replace placeholders <<<
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"  # PASTE YOUR ACTUAL GEMINI KEY HERE
os.environ["SERPER_API_KEY"] = "YOUR_SERPER_API_KEY"  # PASTE YOUR ACTUAL SERPER KEY HERE


# --- Verification ---
# Checks if the keys seem to be loaded (not foolproof, but catches placeholders).
gemini_key_check = os.environ.get("GEMINI_API_KEY")
serper_key_check = os.environ.get("SERPER_API_KEY")

# Basic validation
if not gemini_key_check or gemini_key_check == "YOUR_GEMINI_API_KEY":
    print("⚠️ ERROR: Gemini API Key not found or is still the placeholder value.")
    print("       Please uncomment the line in Option 2 and paste your key inside the quotes.")
elif not serper_key_check or serper_key_check == "YOUR_SERPER_API_KEY":
    print("⚠️ ERROR: Serper API Key not found or is still the placeholder value.")
    print("       Please uncomment the line in Option 2 and paste your key inside the quotes.")
else:
    print("✅ --- API Keys Seem Loaded via Option 2 (Please ensure they are correct and active) ---")

In [ ]:
# Cell 3: Import Necessary Libraries
# Imports required modules from CrewAI, LangChain, Google, and standard libraries.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cell 2

import warnings
import os
import traceback # For detailed error reporting
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI # LangChain wrapper for Gemini

print("--- Libraries Imported ---")

In [ ]:
# Cell 4: Configure LLM (LangChain Wrapper) and Tools (Force Key Re-Read)
# Initializes the core components: the Language Model and the Web Search Tool.
# Adds extra checks and explicit key passing to debug persistent API key errors.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cells 2-3

import os
import traceback
import warnings
from langchain_google_genai import ChatGoogleGenerativeAI # Ensure import is here

warnings.filterwarnings('ignore') # Suppress common warnings

# Initialize variables
llm = None
web_scraper_tool = None

# --- Configure Language Model (Gemini via LangChain) ---
try:
    print("--- Reading GEMINI_API_KEY from environment *immediately* before LLM setup ---")
    # Force re-read from environment variable set in Cell 2
    gemini_api_key_from_env = os.environ.get("GEMINI_API_KEY")

    # Basic validation and Debug Print
    if not gemini_api_key_from_env or gemini_api_key_from_env == "YOUR_NEW_VALID_GEMINI_KEY" or len(gemini_api_key_from_env) < 10: # Basic length check
         print(f"⚠️ ERROR: GEMINI_API_KEY read from environment appears invalid or placeholder.")
         print(f"   Value found: {gemini_api_key_from_env}")
         raise ValueError("GEMINI_API_KEY environment variable not set correctly or invalid.")
    else:
        # Mask most of the key for security in printing
        masked_key = gemini_api_key_from_env[:4] + "****" + gemini_api_key_from_env[-4:]
        print(f"✅ Key read from environment successfully. Using key: {masked_key}")

    # Specify the desired, VALID Gemini model name
    selected_model = 'gemini-1.5-flash'
    print(f"--- Attempting to configure LangChain LLM wrapper with model: {selected_model} ---")

    # Instantiate the LangChain wrapper, explicitly passing the key we just read
    llm = ChatGoogleGenerativeAI(
        model=selected_model,
        google_api_key=gemini_api_key_from_env, # Explicitly pass the just-read key
        convert_system_message_to_human=True
    )
    print(f"--- LangChain LLM Wrapper Configured (Using Gemini model: {selected_model}) ---")

except Exception as e:
    print(f"🚨 Error configuring LangChain ChatGoogleGenerativeAI: {e}")
    print("🚨 Please ensure:")
    print("   - Your GEMINI_API_KEY in Cell 2 is correct, active, and was read correctly above.")
    print(f"   - The model name '{selected_model}' is valid for your API key.")
    print("   - Billing is enabled in your Google Cloud project if required.")
    print("\n--- LLM Configuration Error Traceback ---")
    traceback.print_exc()
    print("--- End LLM Configuration Error Traceback ---")
    llm = None # Ensure llm is None if setup failed

# --- Configure Web Scraping Tool (Serper) --- (No changes here)
try:
    serper_api_key = os.environ.get("SERPER_API_KEY")
    if not serper_api_key or serper_api_key == "YOUR_VERIFIED_SERPER_KEY": raise ValueError("SERPER_API_KEY invalid.")
    web_scraper_tool = SerperDevTool()
    print("--- Web Scraper Tool (Serper) Initialized ---")
except Exception as e:
    print(f"🚨 Error initializing SerperDevTool: {e}"); traceback.print_exc(); web_scraper_tool = None

# --- Final Initialization Check ---
if not llm:
    print("\n🚨🚨🚨 CRITICAL ERROR: LLM object failed to initialize. Cannot proceed. Review Cell 4 output. 🚨🚨🚨")
elif not web_scraper_tool:
     print("\n🚨🚨🚨 CRITICAL ERROR: Web Scraper tool failed to initialize. Cannot proceed. Review Cell 4 output. 🚨🚨🚨")
else:
    print("--- LLM and Tools Initialization Check Passed ---")

In [ ]:
# Cell 5: Define the Agents (Explicitly Assigning LLM to Each Agent)
# Creates each specialized agent for the Science Department simulation.
# The configured LangChain LLM object ('llm') is passed directly to each agent.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cells 2-4

# --- Verification ---
# Double-check that the LLM and Tool objects from Cell 4 are available.
if 'llm' not in globals() or llm is None:
   raise NameError("LLM object 'llm' was not successfully created in Cell 4. Cannot define agents.")
if 'web_scraper_tool' not in globals() or web_scraper_tool is None:
   raise NameError("Tool 'web_scraper_tool' was not successfully created in Cell 4. Cannot define agents.")

# --- Agent Definitions ---
print("--- Defining Agents and assigning the configured LangChain LLM object ('llm') to each ---")

# Agent for handling user input
query_intake_agent = Agent(
    role='Query Intake and Routing Specialist',
    goal='Refine user queries for clarity and specificity, identifying the core scientific question and relevant domain.',
    backstory="Acts as the gateway to the Science Department simulator. Specializes in interpreting user needs and translating them into precise, actionable prompts for scientific analysis.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=False
)

# Agent for formatting the final output
output_formatting_agent = Agent(
    role='Science Communication Editor',
    goal='Transform technical scientific output into a clear, engaging, and well-structured response using Markdown.',
    backstory="Serves as the final polish stage. Excels at making complex information accessible and presentable, focusing on readability, structure, and appropriate formatting without altering scientific content.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=False
)

# Science Agent: Theoretical Physics
theoretical_physicist_agent = Agent(
    role='Theoretical and Classical Physicist',
    goal='Analyze phenomena using foundational physics principles (classical mechanics, thermo, EM, relativity) and develop theoretical models.',
    backstory="Expert in the macroscopic laws of physics. Provides rigorous analysis based on established theories.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=True, # Can delegate to other specialists
    tools=[web_scraper_tool] # Access to web search
)

# Science Agent: Quantum/Nuclear Physics
quantum_nuclear_physicist_agent = Agent(
    role='Quantum Mechanics and Nuclear Physicist',
    goal='Explain phenomena at atomic and subatomic scales using quantum principles (states, particles, reactions).',
    backstory="Focuses on the quantum realm, including particle interactions, nuclear energy, and wave-particle duality.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=True,
    tools=[web_scraper_tool]
)

# Science Agent: Astrophysics/Astronomy
astrophysicist_astronomer_agent = Agent(
    role='Astrophysicist and Astronomer',
    goal='Investigate cosmic objects and phenomena (stars, galaxies, cosmology), applying physics to understand the universe.',
    backstory="Studies the universe at large scales, from planetary systems to cosmology. Interprets astronomical data.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=True,
    tools=[web_scraper_tool]
)

# Science Agent: Electrical Engineering/Instrumentation
electrical_instrumentation_agent = Agent(
    role='Electrical Engineer and Instrumentation Expert',
    goal='Provide expertise on electrical systems (circuits, sensors, controls) and practical instrument design/limitations.',
    backstory="Bridges theory and practical implementation for electrical and electronic systems. Understands signal processing and sensor technology.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=True,
    tools=[web_scraper_tool]
)

# Science Agent: Mathematics/Statistics
mathematician_statistician_agent = Agent(
    role='Mathematician and Statistician',
    goal='Apply mathematical rigor and statistical methods (modeling, analysis, computation) to support scientific inquiry.',
    backstory="Provides the quantitative foundation for the department. Develops models, analyzes data, assesses uncertainty.",
    llm=llm, # Explicitly assign the configured Gemini LLM object
    verbose=True,
    allow_delegation=True,
    tools=[web_scraper_tool]
)

# --- Confirmation ---
print("--- Agents Defined (LLM Explicitly Assigned to Each Agent) ---")

In [ ]:
# Cell 6: Define the Tasks (Using Context Passing)
# Specifies the individual steps the Crew will undertake to fulfill the user's request.
# Uses the 'context' parameter to pass results from one task to the next.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cells 2-5

# Define the list of potential delegatees for the science task
science_agents = [
    theoretical_physicist_agent,
    quantum_nuclear_physicist_agent,
    astrophysicist_astronomer_agent,
    electrical_instrumentation_agent,
    mathematician_statistician_agent
]

# Task 1: Refine the user's initial query.
intake_task = Task(
    description=(
        "1. Receive the user's input topic: '{user_topic}' and prompt: '{user_prompt}'.\n"
        "2. Analyze for clarity, feasibility, and the core scientific question.\n"
        "3. Rewrite the prompt to be specific, actionable, and clear for analysis.\n"
        "4. Identify the most relevant primary scientific field.\n"
        "5. **Output ONLY the single refined prompt string**."
    ),
    expected_output="A single string containing only the refined, actionable prompt suitable for a science agent.",
    agent=query_intake_agent # Assigns this task to the input agent.
    # Output is implicitly passed to tasks that list 'intake_task' in their context.
)

# Task 2: Perform the core scientific analysis based on the refined prompt.
science_analysis_task = Task(
    description=(
        "Analyze the refined scientific prompt provided **via context from the intake task**. "
        "Perform necessary research, calculations, and reasoning based on your assigned role's expertise. "
        "If the prompt requires knowledge outside your core specialization, you MAY delegate to a more suitable expert from the available team if appropriate for achieving the goal. "
        "If current information, specific data, or recent findings are needed, use the 'SerperDevTool' web scraping tool. Cite external sources clearly if used. "
        "Generate a comprehensive, technically accurate, and well-reasoned response."
    ),
    expected_output=(
        "A detailed, scientifically accurate, and well-reasoned technical response addressing the refined prompt. "
        "If the web scraping tool was used, citations or clear integration of findings should be present. "
        "The output should be suitable for final editing."
    ),
    agent=theoretical_physicist_agent, # Assign to a primary science agent (can delegate).
    context=[intake_task] # Explicitly states this task needs the output context from 'intake_task'.
)

# Task 3: Format the technical response for presentation.
formatting_task = Task(
    description=(
        "Receive the technical scientific response **via context from the analysis task**. Format it for presentation to the end-user.\n"
        "Mandatory formatting requirements:\n"
        "1. Ensure clarity, coherence, and a professional, informative tone.\n"
        "2. Correct minor grammatical errors or awkward phrasing.\n"
        "3. Structure the information logically using paragraphs, headings (if appropriate), and lists.\n"
        "4. Apply Markdown formatting effectively for emphasis (**bold**), key terms (*italics*), lists (bulleted or numbered), and potentially code blocks for equations or formulas.\n"
        "5. Ensure the final output is well-structured, easy to read, and visually appealing.\n"
        "6. CRITICAL: Do NOT add new scientific information or opinions; focus solely on presenting the provided technical content effectively."
    ),
    expected_output=(
        "A polished, well-formatted, and easily understandable version of the scientific response, "
        "using Markdown extensively and effectively for presentation."
    ),
    agent=output_formatting_agent, # Assigns this task to the output agent.
    context=[science_analysis_task] # Explicitly states this task needs the output context from 'science_analysis_task'.
)

print("--- Tasks Defined (Using Context Passing for Dependencies) ---")

In [ ]:
# Cell 7: Create and Configure the Crew
# Assembles the agents and tasks into a cohesive Crew responsible for execution.
# Relies on the LLM assigned individually to each agent in Cell 5.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cells 2-6

# --- Verification ---
# Ensure the base LLM object exists for potential use elsewhere (like Cell 8 direct test if re-enabled)
if 'llm' not in globals() or llm is None:
   raise NameError("LLM object 'llm' from Cell 4 is needed but wasn't created.")

# --- Crew Assembly ---
# Combine all defined agents into a list for the Crew.
all_agents = [
    query_intake_agent,
    theoretical_physicist_agent,
    quantum_nuclear_physicist_agent,
    astrophysicist_astronomer_agent,
    electrical_instrumentation_agent,
    mathematician_statistician_agent,
    output_formatting_agent
]

# Define the sequence of tasks for the Crew to execute.
crew_tasks = [
    intake_task,
    science_analysis_task,
    formatting_task
]

# --- Crew Instantiation ---
# Create the Crew instance. It will manage the sequential execution of tasks using the defined agents.
# The LLM logic is now handled by the individual agents as configured in Cell 5.
print("--- Instantiating Crew (Relying on LLM assigned to individual agents) ---")
science_department_crew = Crew(
    agents=all_agents,
    tasks=crew_tasks,
    process=Process.sequential, # Tasks will run in the order defined in 'crew_tasks'.
    # llm= or llm_config= are omitted here; using agent-specific LLMs.
    verbose=True, # Enable logging of Crew and Agent actions. Set to False to reduce output.
    # memory=True, # Optional: Uncomment to enable memory across tasks/runs.
    # cache=True, # Optional: Uncomment to cache task results.
)

# --- Confirmation ---
print("--- Crew Created (Relying on Agent-Specific LLMs) ---")

In [ ]:
# Cell 8: Kickoff the Simulation (No Direct LLM Test)
# Starts the Crew execution process with the user's input.
# Includes error handling for potential issues during the run.
# RUN THIS CELL AFTER RESTARTING THE RUNTIME and running Cells 2-7

print("\n--- Science Query Solving Mode Initializing ---")
print("Please provide the topic and your initial question/prompt.")

# --- Get User Input ---
try:
    user_topic = input("Enter the topic: ")
    user_prompt = input("Enter your question or prompt: ")
except EOFError:
    # Handles cases where input might fail (e.g., non-interactive environment)
    print("\nWarning: Input stream closed unexpectedly. Using default values for demonstration.")
    user_topic = "General Relativity"
    user_prompt = "Explain the basic concept of spacetime curvature simply."

# Bundle inputs into a dictionary for the kickoff method.
# Keys must match placeholders '{user_topic}' and '{user_prompt}' in the intake_task description.
inputs = {
    'user_topic': user_topic,
    'user_prompt': user_prompt
}

print(f"\n--- Inputs Received ---\nTopic: {user_topic}\nPrompt: {user_prompt}")
print("\n--- Starting Simulation... ---")
print("Note: Verbose output from the Crew and Agents will follow.")

# --- Crew Execution ---
# Verify necessary components (LLM object for agents, tool) were initialized in Cell 4.
# The Crew itself relies on the LLM assigned to agents in Cell 5.
if 'llm' in globals() and llm is not None and 'web_scraper_tool' in globals() and web_scraper_tool is not None:
    try:
        print("\nAttempting crew.kickoff()...")
        # Start the main execution flow.
        result = science_department_crew.kickoff(inputs=inputs)

        # If kickoff completes without throwing an exception:
        print("\n\n--- Simulation Complete ---")
        print("\n--- Final Formatted Output ---")
        # Print the final result produced by the last task (formatting_task).
        print(result)

    except Exception as e:
        # Catch any error during the crew's execution.
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("🚨🚨🚨 Crew Execution Failed During Kickoff! 🚨🚨🚨")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"\nError Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # Displaying the traceback helps pinpoint where in the process the error occurred.
        print("\n--- Full Traceback (Crew Kickoff) ---")
        traceback.print_exc()
        print("--- End Traceback (Crew Kickoff) ---")
        print("\n--- Troubleshooting Hints ---")
        print(" * Review the traceback above for specific error location.")
        print(" * Check the verbose agent output preceding the error for context.")
        print(f" * If '503 Service Unavailable', Google's servers for model '{selected_model}' might be overloaded. Wait and retry Cell 8.")
        print(" * Ensure API keys (Cell 2) are correct, active, and billed if necessary.")
        print(" * Ensure model name in Cell 4 is correct and available.")

else:
    # This message indicates a setup failure in earlier cells.
    print("\n🚨 Cannot start simulation because critical components (LLM or Web Scraper) failed to initialize.")
    print("🚨 Please review the output from Cell 4 for initialization errors and fix them first.")

print("\n--- End of Program ---")